In [19]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests

# Читаем текстовый файл по url-ссылке
data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text

### Задание 1.

Давайте введем понятие частоты употребления отдельного слова в документе (`term frequency`, или `tf`). В нашем случае речь идёт не о документах, а о главах книги (выше мы писали, что в текстовом документе главы разделяются строкой '[new chapter]').

Формула для вычисления `term frequency` для слова `word`:
$$ tf_{word, chapter} = \frac {n_{word, chapter}} {n_{chapter}}$$

где
* ${n_{word, chapter}}$ - сколько раз слово `word` встречается в главе `chapter`,
* $n_{chapter}$ - количество слов в главе `chapter`.


Например, слово `"гостья"` употребляется в 15-ой главе 10 раз (${n_{word, chapter}}$).(кстати, главы у нас нумеруются с 0). Общее количество слов в тексте 15-ой главы - 1359 ($n_{chapter}$). Тогда:

$$ tf_{гостья, 15} = \frac{10}{1359} \approx 0.007358$$

**Задание:**

Напишите программу, которая позволит получать частоту употребления любого заданного слова `target_word` в заданной главе `target_chapter`.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `tf` для всех слов из каждой главы, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**

*Для этого вы можете для каждой главы создать словарь, ключами которого являются слова, а значения - частота употребления этого слова в этой главе*

**Протестируйте работу программы на нескольких словах и главах.**

In [ ]:
text = data.split('\n')
text.remove('')

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее']

# **Задача №1** - решение

#### Разбиение текста на словарь со структурой: {номер главы: [список слов]}

In [22]:
chapters_and_words = {} # Словарь со структурой: {номер главы: список всех слов в главе}
words = [] # список, в который будут добавляться слова конкретной главы
chapter_number = 0

for word in text: 
    words.append(word)
    if word == '[new chapter]':
        words.remove('[new chapter]')
        chapters_and_words[chapter_number] = words
        words = []
        chapter_number += 1

#### Составление частотного словаря со структурой: {номер главы: [[слово, частота встречаемости], [слово, частота встречаемости]...]}

In [23]:
words_frequancy = {} # частотынй словарь со структурой: {номер главы: [[слово, частота встречаеомости в главе], [слово, частота встречаеомости в главе]...]}
frequancy_list = []

for chapter in chapters_and_words:                                  # Берем список слов соответсвующий каждой главе
    unique_words = set(chapters_and_words[chapter])                 # Находим какие слова встречаются в главе (без повторений)
    for word in (unique_words):                                     # Берем слово из множества уникальных слов 
        quantity = chapters_and_words[chapter].count(word)          # Считаем сколько раз это слово встретилось в главе
        frequancy = quantity / len(chapters_and_words[chapter])     # Находим частототу встречаемости 
        word_and_freq = [word, frequancy]                           # Добавляем в список слово и его частоту
        frequancy_list.append(word_and_freq)                        # Заносим данные в список для частотного словаря                                                        

    words_frequancy[chapter] = frequancy_list                       # Добавляем список с парами [слово, частота] в частотный словарь по номеру главы 
    frequancy_list = []                                             # Обнуляем добавляемый список для записи новых пар из следующей главы

#### Функция поиска и вывод запрашиваемых значений из блока 1

In [39]:
def get_word_frequency(dictionary, chapter, word):                  # Обявление функции поиска частоты по названию словаря, номеру главы и искомому слову
    target_list = dictionary[chapter]                               # Выделяем нужный список по номеру главы
    for target in target_list:                                      # Объявляем цикл для поиска нужной пары (target)
        if word == target[0]: return target[1]                      # Если запрашиваемое слово word совпадает с первым элементом в паре (target[0]), возвращаем частоту его встречаемости (target[1])

    return ('Слово не найдено')                                     # Если поиск не дал результатов, возвращаем "Слово не найдено"

#### Ввод искомых параметров и вывод

In [41]:
target_word = 'гостья'
target_chapter = 15


print(f"Частота встречаемости слова '{target_word}' из главы № {target_chapter} составляет {round(get_word_frequency(words_frequancy, target_chapter, target_word), 6)}")

Частота встречаемости слова 'гостья' из главы № 15 составляет 0.007358


### Задание 2.

Пришло время познакомиться с понятием `document frequency`.

`Document frequency` (для удобства сократим до `df`) — это доля документов, в которых встречается искомое слово.

Вычисляется по формуле:

$$ df_{word} = \frac{N_{word}}{N} $$

где
* $N_{word}$ - число документов (глав) содержащих слово `word`,
* $N$ - общее число документов (глав).

Объясним на примере: наш текст состоит из 171 главы ($N$), а слово `"человек"` встречается в 115 главах. Тогда:

$$ df_{человек} = \frac{115}{171} \approx 0.6725$$

**Задание:**

Напишите программу, которая позволит вычислять document frequency для заданного слова `target_word` и выведить результат на экран.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально обобщенной. То есть желательно рассчитать характеристику `df` для всех уникальных слов из книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Подсказка:**
*Для этого вы можете создать словарь, ключами которого являются слова из книги, а значения - доля документов, содержащих эти слова*

**Протестируйте работу программы на нескольких словах**

# **Задача №2** - решение

In [4]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests

# Читаем текстовый файл по url-ссылке
data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text

In [25]:
text = data.split('\n')
text.remove('')
text[:10]

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее']

In [27]:
words_collection = set(text)                          # Cоздаем множество элементов текста
words_collection.remove('[new chapter]')              # Убираем из множества разделитель глав '[new chapter]'
count_of_chapters = text.count('[new chapter]') + 1   # Считаем количество глав (+ 1 - потому что в начале первой главы нет '[new chapter]')

In [29]:
unique_words = set(text)                              # множество всех слов в тексте без повторений
chapters_and_words = {}                               # Словарь со структурой: {номер главы: список всех слов в главе}
words = []                                            # список, в который будут добавляться слова конкретной главы
number_of_chapters = 0


for word in text: 
    words.append(word)
    if word == '[new chapter]':
        words.remove('[new chapter]')
        chapters_and_words[number_of_chapters] = words
        words = []
        number_of_chapters += 1

In [30]:
document_count = {}                                                         # Словарь со структурой: {слово: количество глав с этим словом}

for word in unique_words:                                                   # Берем слово из множества unique_words
    if word not in document_count: document_count[word] = 0                 # Добавляем слово в словарь document_count
    for chapter in chapters_and_words:                                      # Выбираем главу из словаря chapter_and_words по номеру главы (по ключу)
        if word in chapters_and_words[chapter]: document_count[word] +=1    # Проверяем есть ли это слово (word) в этой главе (списке) и если есть увеличиваем количество. 
                                                                            # Далее цикл переходит следующей главе, а по факту пробежки по всем главам переходит к новому слову (word)
                                                                            # Алгоритм долгий, работает за лиеное время О большое

In [32]:
document_frequancy = {}
for word in document_count:
    document_frequancy[word] = document_count.get(word) / number_of_chapters

In [ ]:
target_word = 'аристократ'

if target_word not in document_frequancy : print ('Это слово не найдено')
else: print(f"Доля документов, в которых встречается искомое слово '{target_word}' составляет {round(document_frequancy.get(target_word), 5)}")

Доля документов, в которых встречается искомое слово 'аристократ' составляет 0.01176


### Задание 3

Пришло время дать разъяснения: для чего мы делали вычисления выше и что нас ждет впереди?

> Если какое-то слово часто употребляется в документе, то, вероятно, этот документ что-то рассказывает о предмете/действии, описываемом этим словом. Скажем, если вы читаете книгу, в которой много раз употребляется слово `"заяц"`, то, вероятно, эта книга про зайцев.

> Однако, если вы возьмёте слово `"и"`, то оно будет встречаться почти в каждой книге много раз.

Таким образом, если мы хотим найти наиболее значимые слова в книге, мы, с одной стороны, хотим найти наиболее частые слова, а с другой — убрать те, которые не несут важной информации, так как встречаются везде.

Такая задача хорошо решается с помощью `tf-idf` — статистической метрики для оценки важности слова в тексте. Другими словами, `tf-idf` — это «контрастность» слова в документе (насколько оно выделяется среди других слов).

Формула для вычисления следующая:

`tf-idf = term frequency * inverse document frequency`

* `tf` — это частотность термина, которая измеряет, насколько часто термин встречается в документе.

* `idf` — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.

Чтобы получить `idf`, необходимо поделить 1 на полученную в Задании 2 документную частоту (`df`):

$$idf = \frac{1}{df}$$

Мы будем использовать не сырые значения `idf`, а их логарифмы, то есть $tf * log(idf)$. Сейчас мы не будем заострять внимания на том, почему следует использовать именно логарифм — это долгий разговор. Вернемся к нему, когда будем изучать методы машинного обучения для обработки текстов. Подробнее о `tf-idf` вы можете почитать [здесь](https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/).

В качестве примера измерим `tf-idf` слова `"анна"` в главе 4. Слово `"анна"` встречается в указанной главе 7 раз, при этом в 4 главе 1060 слов, всего же слово `"анна"` упоминается в 32 главах из 171.

Таким образом, `tf-idf` данного слова в данной главе будет равно:

$$tf\_idf_{анна, 4} = tf * log(\frac{1}{df}) = \frac{7}{1060} * log(\frac {171}{32}) \approx 0.011067$$

**Примечание:** здесь используется натуральный логарифм по основанию $e$, однако в общем случае основание логарифма не имеет значения, так как характеристика `tf-idf` используется для сравнения контрастности слов между собой

**Задание**:

Напишите программу, которая позволяет вычислять значение `tf-idf` для заданного слова `target_word` в заданной главе `target_chapter`.

**Дополнительное требование:**

*Пострайтесь сделать программу максимально оптимальной. То есть желательно рассчитать характеристику `tf-idf` для всех слов из каждой главы книги, чтобы впоследствии не было необходимости производить вычисления снова.*

**Протестируйте работу программы на нескольких словах и главах.**